# Efficient Parallel Task Execuition with *ray*
Since *ray.Pool*s appear to be batched and hence always wait for the longest task to finish, we need a more efficient way.

We have seen that policy evaluation doesn't vary in duration so much, so round robin is a good strategy for that use case. However, trajectory evaluation varies greatly. But the evaluation can be executed in a non-functional way. Workers don't need to return the result to the caller. They can instead store the result anywhere. Overall task termination status can be polled in a classical check-and-wait loop. The approach illustrated in this little tutorial takes little more that the average time over all task executions, i.e. it is almost perfectly efficient. Hence we'll implement that approach for the self play actors.

In [1]:
import ray
from alphazero.ray.generic import TaskMonitor, RayFilePickler, SimpleCountingDispatcher
from alphazero.gomoku_game import RandomBoardInitializer, GomokuGame
from alphazero.interfaces import MctsParams, PolicySpec
from alphazero.self_play import SelfPlay
from alphazero.policies.ray_impl import HeuristicRayPolicy
from alphazero.ray.trainer import create_pool, PolicyRef, SelfPlayDelegator

### Some Dummy implementations
Workers pick up tasks from a common dispatcher and provide the result to the collector. The dispatcher acts as common counter, but it could as well enumerate input resources.

In [2]:
import uuid
import time

@ray.remote
class Worker:
    def __init__(self, wid, dispatcher, collector):
        self.wid = wid
        self.collector = collector
        self.dispatcher = dispatcher

    def init(self, *args, **kwargs):
        pass

    def work(self):
        """
        fetch task, report result - until all jobs are done
        """
        while True:
            task = ray.get(self.dispatcher.get_task.remote())
            if task is None:
                break
            seqno, effort = task
            time.sleep(effort)
            the_result = str(uuid.uuid4())
            self.collector.collect.remote(self.wid, seqno, the_result)

@ray.remote
class Collector:

    def __init__(self, monitor):
        self.result = []
        self.monitor = monitor

    def collect(self, wid, seqno, load):
        """
        collect all results
        """
        self.result.append((wid, seqno, load))
        self.monitor.report.remote()

    def get_result(self):
        return self.result


@ray.remote
class Dispatcher:

    def __init__(self, num_tasks):
        self.seqno = 0
        self.num_tasks = num_tasks

    def get_task(self):
        """
        Provide tasks of different 'effort' until exhausted
        """
        self.seqno = self.seqno + 1
        if self.seqno <= self.num_tasks:
            return self.seqno, self.seqno % 4 + 1
        else:
            return None


In [3]:
rctx = ray.init(ignore_reinit_error=True)

2022-08-05 07:02:40,156	INFO services.py:1470 -- View the Ray dashboard at http://127.0.0.1:8265


In [4]:
N_WORKERS = 8
N_TASKS = 32

In [5]:
the_monitor = TaskMonitor.remote()

In [6]:
the_collector = Collector.remote(the_monitor)

$N_t$ tasks, duration 1-4s, average $t=2.5s$, with $N_w$ workers: We expect $N_t/N_w*t$ seconds total duration

In [7]:
the_dispatcher = Dispatcher.remote(N_TASKS)

In [8]:
workers = [Worker.remote(wid, the_dispatcher, the_collector) for wid in range(N_WORKERS)]

In [9]:
for worker in workers:
    worker.work.remote()
from alphazero.utils import Timer

with Timer(verbose=True):
    while True:
        time.sleep(.5)
        status = ray.get(the_monitor.get_status.remote())
        if status == N_TASKS:
            break

elapsed time: 11617.784495 ms


A quick check confirms that actor order is sufficiently random to rule out systematic inefficiencies.

In [10]:
result = ray.get(the_collector.get_result.remote())
result

[(4, 4, '644025c1-2e80-4357-b0e6-ef9bf8e1f07e'),
 (0, 8, '65b56b69-8bf8-4d50-8297-9590059e8507'),
 (5, 1, '0271e85e-0fc9-4e4d-b231-25a986158762'),
 (2, 5, '9eb46cb4-75b4-423b-85b3-f0a6dfd3fa99'),
 (1, 2, '82b17438-1578-45bb-9e86-cc8d112e5561'),
 (6, 6, '4294b09c-3322-4593-b298-54b43e58cd9a'),
 (4, 9, '5f1ad445-80d5-4b03-8707-ab985c6812fe'),
 (2, 12, 'ad3ca603-cf7b-4299-8686-6c85a9ac0b03'),
 (3, 3, '461bed43-f3f4-4582-b2bd-6db9a86b532b'),
 (2, 16, 'ae26341c-d0be-4323-81d5-41f2976cef98'),
 (7, 7, '6297d02e-7c96-421e-b798-55c580c49ae8'),
 (0, 10, '7bf80647-ef45-452d-97ff-76cfc4ed63c8'),
 (1, 13, 'b426c67a-a637-4f2a-940a-0a37d3dda9ff'),
 (0, 20, 'b6d2b06d-771f-4396-a2e3-7baf21b14600'),
 (5, 11, '048190c0-c616-4a26-8c9e-c535843d0397'),
 (3, 17, '4d46bdf2-6b14-411c-bbf3-23287f604119'),
 (6, 14, 'bc8814b8-ee8f-467b-8d3b-df24f1963125'),
 (1, 21, 'd0022c1b-05f1-44e7-8d89-eb5c32bfd21e'),
 (3, 24, '20070a13-6b1b-4385-8ce4-5c8ecc0b744a'),
 (4, 15, '0cb10fdf-749c-48a4-a01f-fdd2a95b041b'),
 (2, 18, 

In [11]:
ray.shutdown()

---
# Collecting worker results in a file

In [12]:
rctx = ray.init(ignore_reinit_error=True)

2022-08-05 07:03:19,665	INFO services.py:1470 -- View the Ray dashboard at http://127.0.0.1:8265


In [13]:
import os
filename = os.getcwd() + "/tmp.pickle"
f = RayFilePickler.remote(filename, 'wb+')

In [14]:
f.write.remote([1,2,3])
f.write.remote([1,2,3]);

In [15]:
f.close.remote();

On the ray dashboard, you can observe that closing the file also removes the actor.

### Check the result

In [16]:
!ls -lt *tmp*

-rw-r--r--  1 wgiersche  staff  44 Aug  5 07:03 tmp.pickle


In [17]:
from pickle import Unpickler
def read_entries(file_name):
    fp = open(file_name, 'rb')
    rows = []
    while True:
        try:
            rows.append(Unpickler(fp).load())
        except EOFError:
            return rows

In [18]:
read_entries(file_name=filename)

[[1, 2, 3], [1, 2, 3]]

### Clean up

In [19]:
!rm -f tmp.pickle

In [20]:
ray.shutdown()

---
# Efficient Parallel SelfPlay

In [21]:
ray.init(ignore_reinit_error=True);

2022-08-05 07:03:50,509	INFO services.py:1470 -- View the Ray dashboard at http://127.0.0.1:8265


In [22]:
filename = os.getcwd() + "/tmp.pickle"
the_writer = RayFilePickler.remote(filename, 'wb+')

In [23]:
the_counter = SimpleCountingDispatcher.remote(2)

### The *Business Logic*

In [24]:
params = MctsParams(
    cpuct = 1.0,
    num_simulations=100,
    model_threshold=.2)

# Number of self-play workers
N_SP = 1

# Number of policy workers
N_P = 1

rbi = RandomBoardInitializer(15, 4, 5, 9, 5, 9)
game = GomokuGame(15, rbi)


## Setting up the zoo of actors
A pool of selp-play actors feeding into a round-robin dispatcher serving a fixed number of policy workers. This fixed number may be necessary when the workers utilize non-shareable  resources like e.g. GPUs.

In [25]:
the_dispatcher = create_pool(num_workers=N_P, policy=HeuristicRayPolicy(),
                         board_size=15, cut_off = 0.5)
selfplays = [SelfPlay.remote(mcts_params=params) for _ in range(N_SP)]
for selfplay in selfplays:
    selfplay.init.remote(15, game, PolicySpec(pool_ref=PolicyRef(the_dispatcher)))

(PolicyWorker pid=26434) 2022-08-05 07:04:06.861151: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
(PolicyWorker pid=26434) To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
(PolicyWorker pid=26434) /Users/wgiersche/.local/share/virtualenvs/DeepGomoku-cXtJ_EtM/lib/python3.10/site-packages/numpy/core/fromnumeric.py:2009: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
(PolicyWorker pid=26434)   result = asarray(a).shape


In [26]:
workers = [SelfPlayDelegator.remote(1, the_writer, the_counter, selfplay) for selfplay in selfplays]

In [27]:
for worker in workers:
    worker.work.remote()

In [28]:
the_writer.close.remote();

In [29]:
!ls -lt *tmp*

-rw-r--r--  1 wgiersche  staff  16436 Aug  5 07:06 tmp.pickle


In [30]:
ray.shutdown()

In [31]:
trajectories = read_entries(filename)

In [32]:
trajectories

[[(array([126,  84, 125,  99], dtype=uint8),
   array([ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
           0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
           0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
           0,  0,  0, 10,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
           0, 15,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
           0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
           0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0, 18,  0,  0,  0,  0,
           0,  0,  0,  0, 36, 61,  0,  0, 64, 36, 12,  0,  0,  0,  0,  0,  0,
           0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
           0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
           0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
           0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
           0,  0,  